In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
from src.utils import load_pickle
from src.dataset_utils import get_dataset
from src.model_utils import get_model, get_model_generations
from src.definitions import LEVELS
from argparse import Namespace
import numpy as np

In [ ]:
model, tokenizer = get_model(
    "Llama-2-13b-chat", None, "int8", torch.device("cuda"), max_num_seqs=64
)

In [ ]:
datasets = [
    "piqa",
    "siqa",
    "commonsense_qa",
    "openbookqa",
    "wiki_qa",
    "gpqa",
    "quality",
    "boolq",
]

results = {}

for dataset_name in datasets:
    for level in LEVELS:
        advocate_level = "assistant"

        dataset = get_dataset(
            args=Namespace(base_data_folder="./data"),
            dataset_name=dataset_name,
            cache_dir=None,
            advocate_level="None",
            return_raw=True,
        )

        advocate_dataset = load_pickle(
            f"data/advocate_data/{dataset_name}/Llama-2-70b-chat/generations_{advocate_level}.pkl"
        )

        dataset_idx = 0
        dataset_advocate_idx = 0

        data = []

        while dataset_advocate_idx < len(advocate_dataset):
            answers = (
                dataset[dataset_idx]["correct_answers"]
                + dataset[dataset_idx]["incorrect_answers"]
            )
            for idx in range(len(answers)):
                advocate_answer = answers[idx]
                generation = advocate_dataset[dataset_advocate_idx]["generation"]
                other_answers = [
                    answer for answer in answers if answer != advocate_answer
                ]

                question = dataset[dataset_idx]["question"]

                data.append(
                    f"Here is an explanation. ```{generation}```. Do you find that this explanation correclt reasons that the answer to the question '{question}' should be '{advocate_answer}'? Answer with just a Yes/No."
                )

                dataset_advocate_idx += 1
            dataset_idx += 1

        conversation_data = [
            {
                "conversation_history": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": x},
                ]
            }
            for x in data
        ]

        generations = get_model_generations(
            model, tokenizer, conversation_data, max_new_tokens=5, do_sample=False
        )

        res = np.mean(
            [
                True if "yes" in x["generation"].lower() else False
                for x in generations
                if any([y in x["generation"].lower() for y in ["yes", "np"]])
            ]
        )

        print(dataset_name, level, res)
        results[f"{dataset_name}_{level}"] = res

print(results)